In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
import pandas as pd
import numpy as np
!{sys.executable} -m pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
  Using cached scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.4 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)


In [2]:
# read TSV file
df = pd.read_csv('protein.tsv', sep='\t', header=0) 

# print all columns
print(df.columns)

# transform Organism column from categorical to numerical
df['Organism'] = df['Organism'].astype('category')

not_null_ec = df[df['EC number'].isnull() == False]  

# we're going to preict EC number based on Sequence, then based on interPro and the n based on both
# we're going to test models based on the following features: Sequence, interPro, Sequence + interPro
# we're going to use the following models: Neural network


Index(['Entry', 'Reviewed', 'Entry Name', 'Protein names', 'Organism',
       'Sequence', 'InterPro', 'EC number'],
      dtype='object')


In [3]:
# split the data into train and test
from sklearn.model_selection import train_test_split
X = not_null_ec['Sequence']
y = not_null_ec['EC number']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# add a column for each letter of the alphabet
import string
alphabet = list(string.ascii_uppercase)
for letter in alphabet:
    not_null_ec[letter] = not_null_ec['Sequence'].str.count(letter)
print(not_null_ec.head())

/tmp/ipykernel_5189/1690574462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_null_ec[letter] = not_null_ec['Sequence'].str.count(letter)
/tmp/ipykernel_5189/1690574462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_null_ec[letter] = not_null_ec['Sequence'].str.count(letter)
/tmp/ipykernel_5189/1690574462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

        Entry  Reviewed   Entry Name  \
0  A0A009IHW8  reviewed  ABTIR_ACIB9   
1  A0A023I7E1  reviewed   ENG1_RHIMI   
2  A0A024B7W1  reviewed   POLG_ZIKVF   
3  A0A024SC78  reviewed  CUTI1_HYPJR   
4  A0A024SH76  reviewed   GUX2_HYPJR   

                                       Protein names  \
0  2' cyclic ADP-D-ribose synthase AbTIR (2'cADPR...   
1  Glucan endo-1,3-beta-D-glucosidase 1 (Endo-1,3...   
2  Genome polyprotein [Cleaved into: Capsid prote...   
3                             Cutinase (EC 3.1.1.74)   
4  Exoglucanase 2 (EC 3.2.1.91) (1,4-beta-cellobi...   

                                            Organism  \
0           Acinetobacter baumannii (strain 1295743)   
1                                  Rhizomucor miehei   
2  Zika virus (isolate ZIKV/Human/French Polynesi...   
3  Hypocrea jecorina (strain ATCC 56765 / BCRC 32...   
4  Hypocrea jecorina (strain ATCC 56765 / BCRC 32...   

                                            Sequence  \
0  MSLEQKKGADIISKILQIQNSIGKTT

/tmp/ipykernel_5189/1690574462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_null_ec[letter] = not_null_ec['Sequence'].str.count(letter)
/tmp/ipykernel_5189/1690574462.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_null_ec[letter] = not_null_ec['Sequence'].str.count(letter)


In [4]:
# look each alphabet column, if the sum of the column is 0, drop it
for letter in alphabet:
    if not_null_ec[letter].sum() == 0:
        not_null_ec = not_null_ec.drop(letter, axis=1)

In [5]:
# add a column thats the length of the sequence
not_null_ec['Sequence_length'] = not_null_ec['Sequence'].str.len()
print(not_null_ec.head())

        Entry  Reviewed   Entry Name  \
0  A0A009IHW8  reviewed  ABTIR_ACIB9   
1  A0A023I7E1  reviewed   ENG1_RHIMI   
2  A0A024B7W1  reviewed   POLG_ZIKVF   
3  A0A024SC78  reviewed  CUTI1_HYPJR   
4  A0A024SH76  reviewed   GUX2_HYPJR   

                                       Protein names  \
0  2' cyclic ADP-D-ribose synthase AbTIR (2'cADPR...   
1  Glucan endo-1,3-beta-D-glucosidase 1 (Endo-1,3...   
2  Genome polyprotein [Cleaved into: Capsid prote...   
3                             Cutinase (EC 3.1.1.74)   
4  Exoglucanase 2 (EC 3.2.1.91) (1,4-beta-cellobi...   

                                            Organism  \
0           Acinetobacter baumannii (strain 1295743)   
1                                  Rhizomucor miehei   
2  Zika virus (isolate ZIKV/Human/French Polynesi...   
3  Hypocrea jecorina (strain ATCC 56765 / BCRC 32...   
4  Hypocrea jecorina (strain ATCC 56765 / BCRC 32...   

                                            Sequence  \
0  MSLEQKKGADIISKILQIQNSIGKTT

In [6]:
ec_dict = {}
for ec in y:
    ec_list = ec.split(';')
    for ec_number in ec_list:
        if ec_number in ec_dict:
            ec_dict[ec_number] += 1
        else:
            ec_dict[ec_number] = 1
            
sorted_ec_dict = sorted(ec_dict.items(), key=lambda x: x[1], reverse=True)
print(sorted_ec_dict)

[('7.1.1.-', 4638), ('3.1.-.-', 4163), ('2.7.7.6', 3978), ('2.1.1.-', 2769), ('2.7.11.1', 2560), ('7.1.2.2', 2146), ('3.4.24.-', 1510), ('1.-.-.-', 1507), ('5.2.1.8', 1489), ('3.6.4.13', 1460), ('3.6.4.12', 1453), ('3.6.5.-', 1441), ('2.3.1.-', 1400), ('7.1.1.2', 1383), ('2.3.2.27', 1367), ('6.3.5.-', 1258), ('3.6.1.-', 1243), ('3.1.26.4', 1143), ('3.4.21.-', 1097), ('4.2.1.33', 1073), ('5.6.1.7', 1060), ('3.6.4.-', 1034), ('6.1.1.20', 1008), ('4.2.1.20', 999), ('2.4.1.-', 974), ('2.1.3.15', 974), ('4.2.1.11', 917), ('2.7.4.3', 895), ('3.5.1.2', 894), ('6.1.1.17', 882), ('6.5.1.2', 876), ('3.1.11.6', 853), ('6.1.1.14', 847), ('2.1.1.199', 840), ('7.4.2.8', 839), ('6.1.1.15', 838), ('1.14.-.-', 831), ('6.3.4.4', 828), ('3.5.1.5', 827), (' 3.5.4.9', 822), ('7.-.-.-', 821), ('6.1.1.11', 820), ('2.7.7.7', 816), ('1.5.1.5', 813), ('2.1.2.1', 797), ('2.5.1.75', 792), ('2.5.1.6', 785), ('2.7.7.8', 783), ('3.1.1.29', 780), ('2.3.1.234', 772), ('3.6.-.-', 768), ('6.1.1.4', 764), ('4.2.1.59', 76

In [7]:
# create a new column called EC_711 where it's 1 if the protein has EC number 7.1.1 and 0 otherwise
not_null_ec['EC_711'] = not_null_ec['EC number'].str.contains('7.1.1.-').astype(int)
not_null_ec['EC_31'] = not_null_ec['EC number'].str.contains('3.1.-.-').astype(int)
not_null_ec['EC_2776'] = not_null_ec['EC number'].str.contains('2.7.7.6').astype(int)
not_null_ec['EC_211'] = not_null_ec['EC number'].str.contains('2.1.1.-').astype(int)
not_null_ec['EC_27111'] = not_null_ec['EC number'].str.contains('2.7.11.1').astype(int)

In [8]:
X = not_null_ec.drop(['Entry','Reviewed','Entry Name','Protein names','Organism','Sequence','InterPro', 'EC number','EC_711','EC_31','EC_2776','EC_211','EC_27111'], axis=1)
y_711 = not_null_ec['EC_711']
y_31 = not_null_ec['EC_31']
y_2776 = not_null_ec['EC_2776']
y_211 = not_null_ec['EC_211']
y_27111 = not_null_ec['EC_27111']

In [9]:
#print all columns for X
print(X.columns)

Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O',
       'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'Sequence_length'],
      dtype='object')


In [18]:
# we will build 5 lSTM models, one for each EC number
# we will use the following features: X

# split the data into train and test
from sklearn.model_selection import train_test_split
X_711_train, X_711_test, y_711_train, y_711_test = train_test_split(X, y_711, test_size=0.2, random_state=42)
X_31_train, X_31_test, y_31_train, y_31_test = train_test_split(X, y_31, test_size=0.2, random_state=42)
X_2776_train, X_2776_test, y_2776_train, y_2776_test = train_test_split(X, y_2776, test_size=0.2, random_state=42)
X_211_train, X_211_test, y_211_train, y_211_test = train_test_split(X, y_211, test_size=0.2, random_state=42)
X_27111_train, X_27111_test, y_27111_train, y_27111_test = train_test_split(X, y_27111, test_size=0.2, random_state=42)

In [ ]:
# Building the 711 model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, BatchNormalization

# First LSTM model
model_711 = Sequential()
model_711.add(LSTM(100, input_shape=(X_711_train.shape[1],1)))
model_711.add(Dropout(0.2))
model_711.add(Dense(1, activation='sigmoid'))

model_711.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_711.summary())

model_711.fit(X_711_train, y_711_train, epochs=10, batch_size=64, verbose=1)

# test the model
y_711_pred = model_711.predict(X_711_test)
y_711_pred = (y_711_pred > 0.5).astype(int)

from sklearn.metrics import accuracy_score
print("Accuracy: %.2f%%" % (accuracy_score(y_711_test, y_711_pred)*100))

from sklearn.metrics import recall_score
print("Recall: %.2f%%" % (recall_score(y_711_test, y_711_pred)*100))

from sklearn.metrics import precision_score
print("Precision: %.2f%%" % (precision_score(y_711_test, y_711_pred)*100))

# save the model
model_711.save('model_711.h5')

Accuracy: 99.62% \
Recall: 91.26% \
Precision: 86.30%

In [ ]:
# Building the 31 model

model_31 = Sequential()
model_31.add(LSTM(100, input_shape=(X_31_train.shape[1],1)))
model_31.add(Dropout(0.1))
model_31.add(Dense(1, activation='sigmoid'))

model_31.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_31.summary())

model_31.fit(X_31_train, y_31_train, epochs=10, batch_size=64, verbose=1)

# test the model
y_31_pred = model_31.predict(X_31_test)
y_31_pred = (y_31_pred > 0.3).astype(int)

from sklearn.metrics import accuracy_score, recall_score, precision_score
print("Accuracy: %.2f%%" % (accuracy_score(y_31_test, y_31_pred)*100))
print("Recall: %.2f%%" % (recall_score(y_31_test, y_31_pred)*100))
print("Precision: %.2f%%" % (precision_score(y_31_test, y_31_pred)*100))

model_31.save('model_31.h5')

Accuracy: 98.98% \
Recall: 51.81% \
Precision: 77.01%

In [ ]:
# Building the 2776 model

model_2776 = Sequential()
model_2776.add(LSTM(100, input_shape=(X_2776_train.shape[1],1)))
model_2776.add(Dropout(0.1))
model_2776.add(Dense(1, activation='sigmoid'))

model_2776.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_2776.summary())

model_2776.fit(X_2776_train, y_2776_train, epochs=10, batch_size=64, verbose=1)

# test the model
y_2776_pred = model_2776.predict(X_2776_test)
y_2776_pred = (y_2776_pred > 0.5).astype(int)

from sklearn.metrics import accuracy_score, recall_score, precision_score
print("Accuracy: %.2f%%" % (accuracy_score(y_2776_test, y_2776_pred)*100))
print("Recall: %.2f%%" % (recall_score(y_2776_test, y_2776_pred)*100))
print("Precision: %.2f%%" % (precision_score(y_2776_test, y_2776_pred)*100))

model_2776.save('model_2776.h5')

Accuracy: 99.16% \
Recall: 60.37% \
Precision: 84.10%

In [ ]:
# Building the 211 model

model_211 = Sequential()
model_211.add(LSTM(100, input_shape=(X_211_train.shape[1],1)))
model_211.add(Dropout(0.2))
model_211.add(Dense(1, activation='sigmoid'))

model_211.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_211.summary())

model_211.fit(X_211_train, y_211_train, epochs=10, batch_size=64, verbose=1)

# test the model
y_211_pred = model_211.predict(X_211_test)
y_211_pred = (y_211_pred > 0.5).astype(int)

from sklearn.metrics import accuracy_score, recall_score, precision_score
print("Accuracy: %.2f%%" % (accuracy_score(y_211_test, y_211_pred)*100))
print("Recall: %.2f%%" % (recall_score(y_211_test, y_211_pred)*100))
print("Precision: %.2f%%" % (precision_score(y_211_test, y_211_pred)*100))

model_211.save('model_211.h5')

Accuracy: 99.21% \
Recall: 26.01% \
Precision: 92.50%

In [ ]:
# Building the 27111 model

model_27111 = Sequential()
model_27111.add(LSTM(100, input_shape=(X_27111_train.shape[1],1)))
model_27111.add(Dropout(0.2))
model_27111.add(Dense(1, activation='sigmoid'))

model_27111.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_27111.summary())

model_27111.fit(X_27111_train, y_27111_train, epochs=10, batch_size=64, verbose=1)

# test the model
y_27111_pred = model_27111.predict(X_27111_test)
y_27111_pred = (y_27111_pred > 0.5).astype(int)

from sklearn.metrics import accuracy_score, recall_score, precision_score
print("Accuracy: %.2f%%" % (accuracy_score(y_27111_test, y_27111_pred)*100))
print("Recall: %.2f%%" % (recall_score(y_27111_test, y_27111_pred)*100))
print("Precision: %.2f%%" % (precision_score(y_27111_test, y_27111_pred)*100))

model_27111.save('model_27111.h5')

Accuracy: 99.00% \
Recall: 33.89% \
Precision: 48.40%